# Imports

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta

import inflection
import math

import seaborn as sns
import matplotlib.pyplot as plt 

from IPython.core.display import HTML
from IPython.display import Image

from scipy import stats
from pycorrcat.pycorrcat import corr_matrix

from boruta import BorutaPy

from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.linear_model import LinearRegression, RANSACRegressor, Lasso
from sklearn.model_selection import TimeSeriesSplit

import xgboost as xgb
import pickle

import optuna

import warnings

warnings.filterwarnings('ignore')
#sns.set_theme()

# Funções

In [ ]:
# função padroniza a exibição dos gráficos
def jupyter_settings():
    %matplotlib inline

    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [20, 10]
    plt.rcParams['font.size'] = 24

    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )

    sns.set()
    
jupyter_settings()


# função que calcula e retorna a correlação de Cramer's V
def cramer_v(x , y):
    
    tabela = pd.crosstab(x, y).values
    n = tabela.sum()
    r,k = tabela.shape
    
    kcorr = k - ((k-1)**2 / (n-1 ))
    rcorr = r - ((r-1)**2 / (n-1 ))
    
    chi2 = stats.chi2_contingency(tabela)[0]
    chi2corr = max(0, chi2 - ( (k-1) * (r-1) / (n-1) ))
    
    return np.sqrt((chi2corr / n) / ( min(kcorr-1, rcorr-1) ))


# função que calcula e retorna as métricas do modelo preditivo
def ml_error(model, y, yhat):
    
    mae = mean_absolute_error(y, yhat)
    mape = mean_absolute_percentage_error(y, yhat)
    rmse = np.sqrt(mean_squared_error(y, yhat))
    
    return pd.DataFrame({'Modelo': model,
                       'MAE': mae,
                        'MAPE': mape,
                        'RMSE': rmse}, index = [0])


# função de modelagem preditiva com cross-validation aplicada a time series
def cross_validation_timeseries(name, model, split, data, opt = False):
    
    tscv = TimeSeriesSplit(n_splits = split)
    evaluation = pd.DataFrame()
    
    for train_index, test_index in tscv.split(data):
        
        # split treino e teste
        cv_train, cv_test = data.iloc[train_index], data.iloc[test_index]
        
        #modelo
        cv_model = model.fit(cv_train.drop(['sales', 'date'], axis = 1), cv_train['sales'])

        # predição
        prediction = cv_model.predict(cv_test.drop(['sales', 'date'], axis = 1))
        

        # concatena os resultados a casa iteração
        results = ml_error('CV Linear', np.expm1(cv_test['sales']), np.expm1(prediction))
        evaluation = pd.concat([evaluation, results])
        
    if opt:

        return - evaluation['RMSE'].mean()

    else:  
            # retorna a média e desvio padrão dos resultados    
        return pd.DataFrame({'Modelo': name,
                           'MAE': round(evaluation['MAE'].mean(),2).astype(str) + ' +/- ' + round(evaluation['MAE'].std(),2).astype(str),
                            'MAPE': round(evaluation['MAPE'].mean(),2).astype(str) + ' +/- ' + round(evaluation['MAPE'].std(),2).astype(str),
                            'RMSE': round(evaluation['RMSE'].mean(),2).astype(str) + ' +/- ' + round(evaluation['RMSE'].std(),2).astype(str)}, 
                            index = [0])


# Descrição dos dados

Nesta seção, realizamos uma análise inicial dos dados com o objetivo de entender sua natureza e realizar alguns tratamentos necessários. Através dessa análise, podemos criar hipóteses e posteriormente realizar uma análise exploratória melhor.

## Carregando os dados

Importando os dados de treino com dados de venda

In [ ]:
df_sales = pd.read_csv('data/train.csv', low_memory = False)
df_sales.head()

Importando dados das lojas

In [ ]:
df_store = pd.read_csv('data/store.csv', low_memory = False)
df_store.head()

In [ ]:
df_raw = pd.merge(df_sales, df_store, how = 'left', on = 'Store')
df_raw.head()

## Renomeando Colunas

In [ ]:
df1 = df_raw.copy()

In [ ]:
# usando a função underscore do módulo inflection deixando todas as primeiras letras das palavas em minúsculo
cols_pre = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
            'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
            'CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek','Promo2SinceYear', 'PromoInterval'] 

snakecase = lambda x: inflection.underscore(x) 

cols_new = list(map(snakecase, cols_pre))
df1.columns = cols_new
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.info()

Transformando a coluna date em formato de data.

In [ ]:
df1['date'] = pd.to_datetime(df1['date'])
df1['date'].info()

## Tratamento Valores Nulos

In [ ]:
df1.isnull().sum()

A variável competition_distance representa a distância da loja concorrente mais próxima. Vamos considerar que as lojas com valores nulos não têm concorrentes próximos ou estão muito distantes para serem considerados competidores. Para preencher esses valores nulos, usaremos um número bem acima do maior valor encontrado nesta coluna. Será colocado o dobro do maior valor.

In [ ]:
#competition_distance  
max_value = df1['competition_distance'].max()

df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 2*(max_value) if math.isnan(x) else x)

As variáveis competition_open_since_month e competition_open_since_year são as datas de abertura da loja concorrente, alguns valores nulos podem corresponder a ausência de concorrente próximo, mas, como há um valor elevado ausentes, podem não haver este registro. Mas, para manter esta variável e verificar sua importância no modelo os valores nulos serão preenchidos com a mesma data da coluna 'date' na respectiva linha.

In [ ]:
#competition_open_since_month    
df1['competition_open_since_month'] = list(map(lambda x: x[0].month if math.isnan(x[1]) else x[1], 
                                         df1[['date', 'competition_open_since_month']].values))


#competition_open_since_year     
df1['competition_open_since_year'] = list(map(lambda x: x[0].year if math.isnan(x[1]) else x[1], 
                                         df1[['date', 'competition_open_since_year']].values))

As variáveis Promo2_since_week e Promo2_since_year  descrevem a semana/ano em que a loja participou da promo2. Promo2 representa se a loja participou da continuação da promoção. Verificando os casos nulos estes representam que a loja não participou. Então, semelhante aos casos anteriores, estas linhas serão preenchidas com a semana/ano com base na coluna 'date'.

In [ ]:
#promo2_since_week               
df1['promo2_since_week'] = list(map(lambda x: x[0].week if math.isnan(x[1]) else x[1], 
                            df1[['date', 'promo2_since_week']].values))

#promo2_since_year               
df1['promo2_since_year'] = list(map(lambda x: x[0].year if math.isnan(x[1]) else x[1], 
                            df1[['date', 'promo2_since_year']].values))
             

A variavel promo_interval descreve os meses em que a promo2 ficou ativa. Os valores nulos serão preenchidos com 0 e será criada uma outra variável que observará se a data (date) foi está no período da promo_interval, indicando uma compra na promoção.

In [ ]:
#promo_interval 
df1['promo_interval'].fillna(0, inplace = True)

# criando uma variável com o mês de date
df1['month_date'] = df1.apply(lambda x: x['date'].strftime('%b'), axis = 1)

df1['is_promo'] = list(map(lambda x: 1 if (x[1] !=0) and (x[0] in x[1]) else 0, 
                            df1[['month_date', 'promo_interval']].values))

In [ ]:
df1.isna().sum()

In [ ]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)
df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)

In [ ]:
df1.dtypes

## Análise descritiva

### Variáveis Númericas

In [ ]:
num_attributes = df1.select_dtypes('number')

num_attributes.agg(["mean","median","std","min","max",
                    "skew","kurtosis"]).T.reset_index().rename(columns={'index': 'columns'})

In [ ]:
sns.histplot(df1['sales'], kde = True, bins = 50);

A variável resposta (sales) apresenta muitos valores zero (0), que são os dias em que a loja não abriu, trataremos isto no feature engineering.  
As variáveis de ano, mês e semana transformaremos em uma única variável data e da variável date construiremos outras variáveis para extrair informações e testar hipóteses.

### Variáveis Categóricas

In [ ]:
cat_attributes = df1.select_dtypes('object')
cat_attributes.describe().T

In [ ]:
df_aux = df1[(df1['sales'] > 0)]
plt.subplots(figsize = (10,4))

plt.subplot(1,3,1)
sns.boxplot(x= 'state_holiday', y = 'sales', data = df_aux);
plt.tight_layout()

plt.subplot(1,3,2)
sns.boxplot(x= 'store_type', y = 'sales', data = df_aux);
plt.tight_layout()


plt.subplot(1,3,3)
sns.boxplot(x= 'assortment', y = 'sales', data = df_aux);
plt.tight_layout()


Transformaremos algumas destas variáveis no feature engineering inserindo seus rótulos correspondentes para melhor análise exploratória.

# Feature Engineering

Nesta etapa, são realizadas as transformações e criações de variáveis com base na descrição inicial dos dados e no mapa mental de hipóteses.

### Mapa mental de Hipóteses

In [ ]:
Image('images/mindmap.png')

### Criação das Hipóteses

Hipóteses criadas a partir do mapa mental.

### Loja

***1 -*** Lojas com mais funcionários vendem mais  
***2 -*** Lojas com maior estoque vende mais  
***3 -*** Lojas com maior porte vendem mais  
***4 -*** Lojas com maior variedade de produtos vendem mais  
***5 -*** Lojas com competidores mais próximos vendem menos  
***6 -*** Lojas com competidores há mais tempo vendem mais  

### Produto

***1 -*** Lojas que investem mais em marketing vendem mais  
***2 -*** Lojas que expõe mais os produtos nas vitrines vendem mais  
***3 -*** Lojas com preços menores nos produtos vendem mais  
***4 -*** Lojas com promoções maiores vendem mais  
***5 -*** Lojas com promoções ativas por mais tempo vendem mais  
***6 -*** Lojas com mais promoções consecutivas vendem mais

### Tempo

***1 -*** Lojas com mais feriados vendem menos   
***2 -*** Lojas vende mais no Natal    
***3 -*** Lojas vendem mais ao longo dos anos  
***4 -*** Lojas vendem mais no ssegundo semestre  
***5 -*** Lojas vendem menos nos finais de semana

### Lista Final Hipóteses

Hipóteses filtradas a partir dos dados existentes:

* Lojas com maior variedade de produtos vendem mais    
* Lojas com competidores mais próximos vendem menos  
* Lojas com competidores há mais tempo vendem mais  
* Lojas com promoções ativas por mais tempo vendem mais  
* Lojas com mais promoções consecutivas vendem mais
* Lojas com mais feriados vendem menos   
* Lojas vende mais no Natal
* Lojas vendem mais ao longo dos anos  
* Lojas vendem mais no segundo semestre  
* Lojas vendem menos nos finais de semana

In [ ]:
df2 = df1.copy()

Extraindo os tempos necessários da variável date.

In [ ]:
df2['year'] = df2['date'].dt.year
df2['month'] = df2['date'].dt.month
df2['day'] = df2['date'].dt.day
df2['week_year'] = df2['date'].dt.isocalendar().week
df2['year_and_week'] = df2['date'].dt.strftime('%Y-%W')

Criando variáveis para contagem de tempo de competição

In [ ]:
df2['competition_since'] = list(map(lambda x : datetime(year = x[0], 
                        month = x[1], day = 1), 
                        df2[['competition_open_since_year', 'competition_open_since_month']].values))

df2['competition_time_month'] = ((df2['date'] - df2['competition_since']) / 30).apply(lambda x: x.days)

Criando variáveis para contagem de tempo de promoção.

In [ ]:
df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-'+ df2['promo2_since_week'].astype(str)
df2['promo_since'] = list(map(lambda x: datetime.strptime(x + '-1', '%Y-%W-%w') - timedelta(days = 7), 
                df2['promo_since']))

df2['promo_time_week'] = ((df2['date'] - df2['promo_since']) / 7).apply(lambda x: x.days)

Colocando rótulos para variáveis categóricas.

In [ ]:
#assortiment
df2['assortment_label'] = df2['assortment'].apply(lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extend')

#state_holiday
df2['state_holiday'] = df2['state_holiday'].apply(lambda x: 'public_holiday' if x == 'a' 
                        else 'easter_holiday' if x == 'b' else 'christmas' if x == 'c' else 'regular_day')

In [ ]:
df2.head().T

# Filtragem Variáveis

Com base em regras de negócio e restrições específicas, algumas linhas e variáveis serão removidas.  
A variável 'customers' não estará disponível para previsões com novos dados e, portanto, será excluída.   
Além disso, lojas fechadas não realizam vendas, então essas linhas serão removidas, assim como as vendas com valor zero.  
Também serão excluídas algumas colunas que foram transformadas no Feature Engineering.

In [ ]:
df3 = df2.copy()

## Filtragem de linhas

In [ ]:
df3 = df3[(df3['open'] != 0) & (df3['sales'] > 0)]

## Seleção de colunas

In [ ]:
cols_drop = ['customers', 'open', 'promo_interval', 'month_date']
df3 = df3.drop(cols_drop, axis = 1)

# Análise Exploratória de Dados

In [ ]:
df4 = df3.copy()

## Análise Univariada

### Variável resposta

In [ ]:
sns.histplot(df4['sales'], kde = True, bins = 50);

A variável sale apresenta uma distribuição com assimetria a esquerda, os valores se acumulam nos valores menores.

### Variáveis Numéricas

In [ ]:
num_attributes = df4.select_dtypes('number')
num_attributes.hist(bins = 25);
plt.tight_layout()

As vendas durante a semana são constantes, apenas domingo (dia 7 da semana) é bem pouco devido a ser raro as lojas abrirem.  
A maioria das lojas possuem competidores próximos.  
Os competidores abrem mais lojas nos meses 9 e 4.  
As lojas tiveram mais competidores no último ano.  
Há maior quantidade de vendas quando não tem promoção.  
Há maior quantidade de vendas nos primeiros 7 meses, nos últimos meses tem constancia menor de vendas.  
Houve um pico de vendas nas promoçoes de 2013 e após constante queda até 2015.

### Variáveis categóricas

In [ ]:
df4.select_dtypes('object').columns

In [ ]:
holiday = df4[(df4['state_holiday'] != 'regular_day')]

plt.subplot(221)
sns.countplot(x = holiday['state_holiday']);

# plt.subplot(322)
# sns.kdeplot(data = holiday, x = 'sales', hue = 'state_holiday', fill = True)
# plt.title('Sales holiday');

plt.subplot(222)
sns.countplot(x = df4['store_type']);

# plt.subplot(324)
# sns.kdeplot(data = df4, x = 'sales', hue = 'store_type', fill = True)
# plt.title('Sales store type')

plt.subplot(223)
sns.countplot(x = df4['assortment_label']);

# plt.subplot(326)
# sns.kdeplot(data = df4, x = 'sales', hue = 'assortment_label', fill = True);
# plt.title('Sales assortment');
# plt.tight_layout();

## Análise Bivariada

Análise das variáveis buscando validar as hipóteses anteriormente criadas

#### H1. Lojas com maior variedade de produtos vendem mais.
_**Não validada, lojas com maior variedade(assortment) vendem menos.**_

In [ ]:
aux1 = df4[['sales', 'assortment_label']].groupby('assortment_label').sum().reset_index()

plt.subplot(121)
sns.barplot(x = 'assortment_label', y = 'sales', data = aux1);

plt.subplot(122)
sns.kdeplot(data = df4, x = 'sales', hue = 'assortment_label', fill = True);


In [ ]:
aux2 = df4[['year_and_week','sales', 'assortment_label']].groupby(['year_and_week','assortment_label']).sum().reset_index()
aux2.pivot(index = 'year_and_week', columns = 'assortment_label', values = 'sales').plot();

aux3 = aux2[aux2['assortment_label'] == 'extra']
aux3.pivot(index = 'year_and_week', columns = 'assortment_label', values = 'sales').plot();

Considerando 'extra' como lojas de maior variedade/sortimento a hipótese nao é validada, este tipo de loja vende menos.  
Basic e extends são semelhantes em vendas e se comportam igualmente ao longo do tempo. O grupo mais distinto e com poucas vendas é o 'extra', observando-o ao longo do tempo, a tendência não é linear como parece mostrar no primeiro gráfico. Parece haver uma sazonalidade onde se vende menos neste tipo de loja. 

#### H2. Lojas com competidores mais próximos vendem menos.
_**Não validada, lojas com competidores mais próximos vendem mais.**_

In [ ]:
aux1 = df4[['sales', 'competition_distance']].groupby('competition_distance').sum().reset_index()

plt.subplot(131)
sns.scatterplot(x = 'competition_distance', y = 'sales', data = aux1);

plt.subplot(132)

bins = list(np.arange(0,20000,1000))
aux1['binned_competition_distance'] = pd.cut(aux1['competition_distance'], bins = bins)
aux2 = aux1[['sales', 'binned_competition_distance']].groupby('binned_competition_distance').sum().reset_index()

sns.barplot(x = 'binned_competition_distance', y = 'sales', data = aux2);
plt.xticks(rotation = 45);

plt.subplot(133)
sns.heatmap(aux1[['sales', 'competition_distance']].corr(method = 'pearson'), annot = True);

Os gráficos mostram o contrário da hipótese, lojas com competidores mais próximos tendem a vender mais, e a correlação mostra esta tendência decrescente. 

#### Lojas com competidores há mais tempo vendem mais.
_**Não validada, lojas com competidores há mais tempo vendem menos.**_

In [ ]:
aux1 = df4[['sales', 'competition_time_month']].groupby('competition_time_month').sum().reset_index()
aux2 = aux1[(aux1['competition_time_month'] < 120) & (aux1['competition_time_month'] != 0)]

plt.subplot(211)
sns.barplot(x = 'competition_time_month', y = 'sales', data = aux2);
plt.xticks(rotation = 90);

plt.subplot(223)
sns.regplot(x = 'competition_time_month', y = 'sales', data = aux2);

plt.subplot(224)
sns.heatmap(aux1.corr(method = 'pearson'), annot = True);

Como comparamos a data da venda com a data de abertura do concorrente há valores negativos pois alguns competidores abriram loja depois da data de venda. Podemos observar que quanto mais próximo de 0, ou seja, mais recente a competição, maior são as vendas. A competição tem bastante impacto nas vendas, podemos notar uma tendência decrescente mas não é uma correlação linear.

#### Lojas com promoções ativas por mais tempo vendem mais .
_**Não validada, lojas com promoções ativas tem uma regularidade de vendas durante um tempo, após, começa a declinar.**_

In [ ]:
aux1 = df4[['promo_time_week', 'sales']].groupby('promo_time_week').sum().reset_index()
aux2 = aux1[aux1['promo_time_week'] > 0] # tempo promoção extendida
 

bins = list(np.arange(0,320,5))
aux2['binned_promo_time'] = pd.cut(aux1['promo_time_week'], bins = bins)
aux3 = aux2[['sales', 'binned_promo_time']].groupby('binned_promo_time').sum().reset_index()

aux4 = aux1[aux1['promo_time_week'] < 0] # tempo promoção regular


plt.subplot(221)
sns.barplot(x = 'binned_promo_time', y = 'sales', data = aux3);
plt.title('Promoção extendida', fontsize = 20)
plt.xticks(rotation = 90);

plt.subplot(222)
sns.regplot(x = 'promo_time_week', y = 'sales', data = aux2);

plt.subplot(223)
sns.barplot(x = 'promo_time_week', y = 'sales', data = aux4);
plt.title('Promoção regular', fontsize = 20)
plt.xticks(rotation = 90);

plt.subplot(224)
sns.regplot(x = 'promo_time_week', y = 'sales', data = aux4);
plt.tight_layout();

Na promoção estendida, percebe-se que há um período de efeito em que as vendas atingem um patamar e mantêm-se regulares, mas começa a declinar após a semana 225, o que gera uma reta com tendência negativa. Já na promoção regular, ocorrem saltos de vendas à medida que a promoção estendida se aproxima mostrando um tendência crescente. Portanto, a hipótese de que as vendas aumentam à medida que a promoção se estende não é validada, já que as vendas caem após um período de regularidade.

In [ ]:
plt.figure(figsize = (8,4))
sns.heatmap(aux1.corr(method = 'pearson'), annot = True);

Esta variável apresenta correlação negativa muito baixa, possivelmente afetada pelo período longo de constância na promoção, talvez não seja relevante para o modelo se não for combinada com outra(s) variável(is).

#### Lojas com promoções consecutivas vendem mais.
_**Não validada, lojas com promoção estendida venderam menos.**_

In [ ]:
df4[['promo', 'promo2', 'sales']].groupby(['promo', 'promo2']).sum().reset_index().sort_values(by = 'sales', ascending = False)

In [ ]:
aux1 = df4[(df4['promo'] == 1) & (df4['promo2'] == 1)][['year_and_week', 'sales']].groupby('year_and_week').sum().reset_index()
ax = aux1.plot();

aux2 = df4[(df4['promo'] == 1) & (df4['promo2'] == 0)][['year_and_week', 'sales']].groupby('year_and_week').sum().reset_index()
aux2.plot(ax =ax);

ax.legend(labels = ['Tradicional & Estendida', 'Tradicional']);

Como se observa no gráfico a prorrogação da promoção não aumentam as vendas, contrariando a hipótese. E se percebe nas linhas do gráfico, que na maior parte o comportamento é o mesmo, possivelmente não será uma variável tão relevante para o modelo.

#### Lojas vende mais no Natal
_**Não validada, as lojas vendem menos no feriado de Natal.**_

In [ ]:
aux1 = df4[df4['state_holiday'] != 'regular_day']
aux2 = aux1[['state_holiday', 'sales']].groupby('state_holiday').sum().reset_index()

plt.subplot(121)
sns.barplot(x = 'state_holiday', y = 'sales', data = aux2);

plt.subplot(122)
aux3 = aux1[['year', 'state_holiday', 'sales']].groupby(['year', 'state_holiday']).sum().reset_index()
sns.barplot(x = 'year', y = 'sales', hue = 'state_holiday', data = aux3);


#### Lojas vendem mais ao longo dos anos  
_**Não validada, as vendas estão com tendência de queda.**_

In [ ]:
df4[['year', 'sales']].groupby('year').sum().plot(kind = 'bar');

Apesar dos dados do último ano (2015) não estar completo, podemos verificar uma tendência de queda nas vendas.

#### Lojas vendem mais no segundo semestre  
_**Não validada, as lojas vendem menos no segundo semestre.**_

In [ ]:
aux1 = df4[['month', 'sales']].groupby('month').sum().reset_index()

plt.subplot(121)
sns.barplot(x = 'month', y = 'sales', data = aux1);

plt.subplot(122)
sns.regplot(x = 'month', y = 'sales', data = aux1);

Nos gráficos, notamos o contrário da hipótese, nos primeiros 6 meses há mais vendas, e uma queda notável a partir do 7º mês.

#### Lojas vendem menos nos finais de semana
_**Validada, as vendas caem nos finais de semana.**_

In [ ]:
aux1 = df4[['day_of_week', 'sales']].groupby('day_of_week').sum().reset_index()

plt.subplot(121)
sns.barplot(x = 'day_of_week', y = 'sales', data = aux1);

plt.subplot(122)
sns.regplot(x = 'day_of_week', y = 'sales', data = aux1);

Os gráficos mostram queda nas vendas ao se aproximar dos fins de semana, assim, as vendas tendem a ser maiores em dias de semana.

## Análise Multivariada

### Variáveis númericas

In [ ]:
corr_num = num_attributes.corr(method = 'pearson')
sns.heatmap(corr_num, annot = True);

### Variáveis categóricas

Para verificar a associação entre as colunas categóricas vamos utilizar o coeficiente V de Cramer com correção (https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V), que pode ser aplicado em situações onde a informação se encontra distribuída por categorias nominais. O coeficiente tem valores entre 0 e 1, quanto mais próximo de 1 maior a correlação.

In [ ]:
cat_attributes = df4.drop(['assortment', 'year_and_week'], axis =1).select_dtypes('object')

In [ ]:
rows= []

for column in cat_attributes.columns:
    col = []
    
    for column2 in cat_attributes.columns:
        if column == column2:
            cramer = 1.0
        else:
            cramer =  corr_matrix(cat_attributes,[column,column2]).values[0][1]
        col.append(round(cramer, 4))
        
    rows.append(col)
     
cramer_results = np.array(rows)

df_cramer = pd.DataFrame(cramer_results, columns = cat_attributes.columns, index =cat_attributes.columns)
sns.heatmap(df_cramer,annot = True) ; 

# Preparação dos Dados

In [ ]:
df5 = df4.copy()

## Rescaling

As variáveis numéricas não possuem uma distribuição normal, então para transformar estes dados vamos utilizar técnicas de rescaling, Min-Max Scaler (Normalização) para variáveis que não são afetadas por outliers e o Robust Scaler para as que possuem outliers relevantes:  
<center><b>Min-Max Scaler:</b></center></br>
$$
X_n = \frac{X_i + X_min}{X_max - X_min)}\\ \\    
$$</br>
$$
\mu = média\\
$$
<center><b>Robust Scaler</b></center></br>
$$
X_n = \frac{X_i + Q_1(X)}{Q_3(X) - Q_1(X)}
$$ </br>
$$
Q_1(X) = 1º Quartil\\
Q_3(X) = 3º Quartil
$$

In [ ]:
rc = RobustScaler()
df5['competition_distance'] = rc.fit_transform(df5[['competition_distance']].values)
df5['competition_time_month'] = rc.fit_transform(df5[['competition_time_month']].values)

mms = MinMaxScaler()
df5['promo_time_week'] = mms.fit_transform(df5[['promo_time_week']].values)
df5['year'] = mms.fit_transform(df5[['year']].values)

## Transformação

### Encoding

In [ ]:
# One Hot Enconding state_holiday
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe_state_holiday = ohe.fit_transform(df5[['state_holiday']].values.reshape(-1,1)).toarray()

for i, column in enumerate(ohe.get_feature_names_out()):
    df5[column] = ohe_state_holiday[:,i]
    

# Label Enconder
le = LabelEncoder()
df5['store_type'] = le.fit_transform(df5['store_type'])

# Ordinal Encoder
assortment_dict = {'basic': 1, 'extra': 2, 'extend': 3}
df5['assortment'] = df5['assortment_label'].map(assortment_dict)

### Transformação da variável resposta

In [ ]:
# transformação logarítma
df5['sales'] = np.log1p(df5['sales'])

### Transformação de Natureza

As variáveis restantes são variáveis de tempo e que denotam um ciclo, para realizar as transformações vamos se basear no círculo trigonométrico, onde cada valor(dia da semana ou mês, por exemplo) será representada por duas novas features com as medidas de seno e outra cosseno, como explica a imagem:

In [ ]:
Image('images/ciclo_mes.jpg')

In [ ]:
# month
df5['month_sin'] = df5['month'].apply(lambda x: np.sin(x * (2. * np.pi/12 )))
df5['month_cos'] = df5['month'].apply(lambda x: np.cos(x * (2. * np.pi/12 )))                                 
                                      
#day_of_week
df5['day_of_week_sin'] = df5['day_of_week'].apply(lambda x: np.sin(x * (2. * np.pi/7 )))
df5['day_of_week_cos'] = df5['day_of_week'].apply(lambda x: np.cos(x * (2. * np.pi/7 )))                                 

#day
df5['day_sin'] = df5['day'].apply(lambda x: np.sin(x * (2. * np.pi/30 )))
df5['day_cos'] = df5['day'].apply(lambda x: np.cos(x * (2. * np.pi/30 )))

#week_year
df5['week_year_sin'] = df5['week_year'].apply(lambda x: np.sin(x * (2. * np.pi/52 )))
df5['week_year_cos'] = df5['week_year'].apply(lambda x: np.cos(x * (2. * np.pi/52 )))


# Seleção de variáveis

In [ ]:
df6 = df5.copy()

## Divisão treino e teste

In [ ]:
df6.head()

In [ ]:
cols_drop = ['week_year', 'day', 'month', 'day_of_week', 'promo_since', 
             'competition_since', 'year_and_week', 'state_holiday', 'assortment_label']

df6 = df6.drop(cols_drop, axis = 1)

In [ ]:
df6.head()

Todas as lojas possuem a mesma data inicial e final de vendas neste dataset, como o modelo preditivo tem um objetivo temporal, vamos separar para o teste as últimas 6 semanas de venda. A última data de vendas é 2015-07-31 assim, a última data para treino será 2015-06-18

In [ ]:
# training dataset
X_train = df6[df6['date'] < '2015-06-19']
y_train = X_train['sales']

# test dataset
X_test = df6[df6['date'] >= '2015-06-19']
y_test = X_test['sales']

print('Treino:','\n','shape: X-->',X_train.shape, 'y-->',y_train.shape)
print('Data min:', X_train['date'].min())
print('Data max:', X_train['date'].max(), '\n')

print('Teste:','\n','shape: X-->',X_test.shape, 'y-->',y_test.shape)
print('Data min:', X_test['date'].min())
print('Data max:', X_test['date'].max())

## Boruta para seleção de variáveis

In [ ]:
X_train_n = X_train.drop(['date', 'sales'], axis = 1).values
y_train_n = y_train.values

In [ ]:
rf = RandomForestRegressor(n_jobs = 1)

boruta = BorutaPy(rf, n_estimators = 'auto', verbose = 2,random_state = 42).fit(X_train_n, y_train_n)

In [ ]:
#cols_sel = boruta.support_.tolist()
cols_sel = [True,True,False, True, True, True, True, True, True, True, True, False, False, True,
 True, False, False, False, False, False, True, True, True, True, True, False, True]

# melhores features
cols_boruta = X_train.drop(['date', 'sales'], axis = 1).iloc[:,cols_sel].columns.to_list()


# features não selecionadas
cols_n_sel = list(np.setdiff1d(X_train.drop(['date', 'sales'], axis = 1).columns, cols_boruta))

# colunas selecionadas
cols_boruta

In [ ]:
# colunas não selecionadas 
cols_n_sel

Das features não selecionadas pelo Boruta vamos manter para o modelo as variáveis month_sin e week_year_sin, são variáveis cíclicas que pela intuição da análise exploratória e hipóteses podem ser relevantes, e,  as outras variáveis que complementam o 'ciclo' foram selecionadas pelo boruta(month_cos e week_year_cos).

In [ ]:
cols_selected = ['store', 'promo', 'store_type', 'assortment', 'competition_distance', 'competition_open_since_month',
 'competition_open_since_year', 'promo2', 'promo2_since_week', 'promo2_since_year', 'competition_time_month', 'promo_time_week', 
'month_cos', 'month_sin','day_of_week_sin', 'day_of_week_cos', 'day_sin', 'day_cos', 'week_year_cos', 'week_year_sin']

# Modelos de machine learning

## Modelo Base

In [ ]:
x_train = X_train[cols_selected]
x_test = X_test[cols_selected]

In [ ]:
aux = x_test.copy()
aux['sales'] = y_test.copy()

aux1 = x_train.copy()
aux1['sales'] = y_train.copy()

# armazenando a media de vendas por loja da base de treino e atribuindoa predição a base de teste
# predição
aux2 = aux1[['sales', 'store']].groupby('store').mean().reset_index().rename(columns = {'sales': 'predict'})
aux3 = pd.merge(aux, aux2, how = 'left', on = 'store')

# performance
model_baseline = ml_error('Average Model', np.expm1(aux['sales']), np.expm1(aux3['predict']))
model_baseline

In [ ]:
df6.head()

## Regressão Linear

In [ ]:
# modelo
lr = LinearRegression().fit(x_train, y_train)


# predição
yhat_lr = lr.predict(x_test)

# performance
lr_result = ml_error('Regressão Linear', np.expm1(y_test), np.expm1(yhat_lr))
lr_result

In [ ]:
columns_add = ['sales', 'date']
cols_full = cols_selected + columns_add
training = X_train[cols_full]

In [ ]:
model = LinearRegression()
linear_cv = cross_validation_timeseries('Linear', model, 5, training )
linear_cv

Este modelo apresenta erros maiores que o modelo base, o que pode ocorrer que os dados não apresentam um comportamento linear ou eventuais outliers estão impactando a performance do modelo. 
Então, analisaremos alguns modelos pensando nestas observações. Algoritmo RASAC modelar detectando e desconsiderando outliers e alguns algoritmos não lineares.

## Regressão Linear RANSAC

In [ ]:
# modelo
lr_ransac = RANSACRegressor(loss = 'squared_error').fit(x_train, y_train)

# predição
yhat_ransac = lr_ransac.predict(x_test)

# performance
lr_ransac_result = ml_error('Regressão RANSAC', np.expm1(y_test), np.expm1(yhat_ransac))
lr_ransac_result

In [ ]:
# verificando a quantidade de outliers detectados
outliers = np.logical_not(lr_ransac.inlier_mask_)
outliers.sum()

In [ ]:
model = RANSACRegressor(loss = 'squared_error')
ransac_cv = cross_validation_timeseries('RANSAC', model, 5, training )
ransac_cv

A retirada de muitas amostras (que foram entendidas pelo modelo como outliers) fez os erros aumentarem. Chegamos na conclusão até aqui que, os outliers não impactam na modelagem e os dados não possuem um comportamento mais complexo, não é linear.

## Random Forest

In [ ]:
# modelo
lr_rf = RandomForestRegressor(n_estimators = 50, n_jobs = -1, random_state = 30).fit(x_train, y_train)

# predição
yhat_rf = lr_rf.predict(x_test)

# performance
lr_rf_result = ml_error('Random Forest', np.expm1(y_test), np.expm1(yhat_rf))
lr_rf_result

In [ ]:
model = RandomForestRegressor(n_estimators = 50, n_jobs = -1, random_state = 30)
rf_cv = cross_validation_timeseries('Random Forest', model, 5, training )
rf_cv

## XGBoost

In [ ]:
# modelo
lr_xgb = xgb.XGBRegressor(objective = 'reg:squarederror',
                          n_estimators = 50,
                          learning_rate = 0.1,
                          max_depth = 10,
                          colsample_bytree = 0.9).fit(x_train, y_train)

# predição
yhat_xgb = lr_xgb.predict(x_test)

# performance
lr_xgb_result = ml_error('XGBoost', np.expm1(y_test), np.expm1(yhat_xgb))
lr_xgb_result

In [ ]:
model = xgb.XGBRegressor(objective = 'reg:squarederror',
                          n_estimators = 50,
                          learning_rate = 0.1,
                          max_depth = 10,
                          colsample_bytree = 0.9)

xgb_cv = cross_validation_timeseries('XGBoost', model, 5, training)
xgb_cv

## Comparação dos modelos

In [ ]:
# modelos simples
models_result = pd.concat([model_baseline, lr_result, lr_ransac_result, lr_rf_result, lr_xgb_result])
models_result.sort_values('RMSE')

In [ ]:
# modelos com cross validation
models_result = pd.concat([linear_cv, ransac_cv, rf_cv, xgb_cv])
models_result.sort_values('RMSE')

O Random Forest obteve melhor performance, assim, vamos utilizá-lo para a próxima etapa, otimização dos hiperparâmetros.

# Otimização dos hiperparâmetros

In [ ]:
n_estimators = n_estimators, 
max_features= max_features,
max_depth = max_depth,
min_samples_split = min_samples_split,
min_samples_leaf = min_samples_leaf,
bootstrap = bootstrap,
n_jobs = -1, random_state = 30

In [ ]:
params = {'n_estimators': [100,500],
         
          'max_depth':[5,20],
          'min_samples_split': [2,10],
          'min_samples_leaf': [1,5], 
}
#'max_features': ['auto', 'sqrt'],
#params = [(50,500),
 #         ('auto', 'sqrt'),
  #        (5,20),
   #       (2,5),
    #      (1,5),
     #     (True, False)]    


def treina_modelo(setup):
    
    #n_estimators = params[0]
    #max_features = params[1]
    #max_depth = params[2]
    #min_samples_split = params[3]
    #min_samples_leaf = params[4]
    #bootstrap =  params[5]  
    
    

    model = RandomForestRegressor(setup)
    
    rmse_cv = cross_validation_timeseries('Random Forest', model, 5, training, True )
    
    return rmse_cv

In [ ]:
from skopt import BayesSearchCV
from bayes_opt import BayesianOptimization, UtilityFunction

In [ ]:
#results = BayesSearchCV(model, params, n_iter = 50, cv = 5, scoring = 'mean_absolute_error')

#optimizer.maximize(ini_points = 5, n_inter = 10)

util = UtilityFunction(kind='ucb',
                                kappa=1.96,
                                xi=0.001)

def func_opt(n_estimators, max_features,max_depth,min_samples_split,min_samples_leaf):
    
    params_model = {}
    params_model['n_estimators'] = int(n_estimators)
    params_model['max_depth'] = int(max_depth)
    params_model['min_samples_split'] = int(min_samples_split)
    params_model['min_samples_leaf'] = int(min_samples_leaf)
       
    model = RandomForestRegressor(n_jobs = -1, random_state = 30, **params_model)
    
    rmse = cross_validation_timeseries('Random Forest', model, 5, training, True )
    
    return -rmse

optimizer = BayesianOptimization(f = func_opt, pbounds = params, verbose = 1, random_state = 30 )
optimizer.maximize(ini_points = 20, n_inter = 4)

In [ ]:
from bayes_opt import BayesianOptimization
from bayes_opt.util import Colours


def rfc_cv(n_estimators, max_features,max_depth,min_samples_split,min_samples_leaf, training):
    
    estimator = RandomForestRegressor(
        n_estimators=int(n_estimators),
        max_features =max_features,
        max_depth = max_depth,
        min_samples_split = int(min_samples_split),
        min_samples_leaf = int(min_samples_leaf),
        random_state=30
    )
    
    rmse = cross_validation_timeseries('Random Forest', estimator, 5, training, True )
    
    return -rmse


def optimize_rfc(training):
    
    def rfc_crossval(n_estimators, max_features,max_depth,min_samples_split,min_samples_leaf):
       
        return rfc_cv(
            n_estimators=int(n_estimators),
            max_features=max(min(max_features, 0.999), 1e-3),
            max_depth = int(max_depth),
            min_samples_split = int(min_samples_split),
            min_samples_leaf = min_samples_leaf,
            training=training
        )

    optimizer = BayesianOptimization(
        f=rfc_crossval,
        pbounds={
           'n_estimators': (100,500),
          "max_features": (0.1, 0.999),
            'max_depth':(5,20),
          'min_samples_split': (2,10),
          'min_samples_leaf': (1,5) 
            },
        random_state=30,
        verbose=1
    )
    optimizer.maximize(n_iter=10)

    print("Final result:", optimizer.max)

if __name__ == "__main__":

    print(Colours.green("--- Optimizing Random Forest ---"))
    optimize_rfc(training)
# aquiiiiiiiiiiiiiiiii

In [ ]:
estimator = RandomForestRegressor(
        n_estimators=485,
        max_features = 'sqrt',
        max_depth = 15,
        min_samples_split = 3,
        min_samples_leaf = 4,
        random_state=30)

estimator_ = cross_validation_timeseries('RFR', estimator, 5, training)
estimator_